# Deconvolution of 10x Visium single cell data

## 1. Load libraries and h5ad file

In [1]:
import scanpy as sc
import squidpy as sq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/Users/thorsten/.pyenv/versions/Spatial_Transcriptomics/lib/python3.13/site-packages/scanpy/_utils/__init__.py:33: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  from anndata import __version__ as anndata_version
/Users/thorsten/.pyenv/versions/Spatial_Transcriptomics/lib/python3.13/site-packages/scanpy/__init__.py:24: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  if Version(anndata.__version__) >= Version("0.11.0rc2"):
/Users/thorsten/.pyenv/versions/Spatial_Transcriptomics/lib/python3.13/site-packages/scanpy/readwrite.py:16: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  if Version(anndata.__version__) >= Version("0.11.0rc2"):
/Users/thorsten/.pyenv/versions/Spatial_Transcriptomics/lib/python3.13/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed 

In [3]:
# Check the path you are working from
!pwd

/Users/thorsten/code/ThorstenCodes/Bioinformatics_TK/Projects/Spatial Transcriptomics/Project_1/dataset2_python/scripts


In [4]:
# Change the path for easier navigation through the project
%cd '/Users/thorsten/code/ThorstenCodes/Bioinformatics_TK/Projects/Spatial Transcriptomics/Project_1/dataset2_python/'

/Users/thorsten/code/ThorstenCodes/Bioinformatics_TK/Projects/Spatial Transcriptomics/Project_1/dataset2_python


In [5]:
# load preprocessed data
sc.read_h5ad('./results/preprocessed.h5ad')

AnnData object with n_obs × n_vars = 1228 × 13081
    obs: 'in_tissue', 'array_row', 'array_col', 'spot', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts', 'n_genes', 'Clusters'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'Clusters', 'Clusters_colors', 'hvg', 'log1p', 'neighbors', 'pca', 'rank_genes_groups', 'spatial', 'spot_colors', 'umap'
    obsm: 'X_pca', 'X_umap', 'spatial'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

/Users/thorsten/code/ThorstenCodes/Bioinformatics_TK/Projects/Spatial Transcriptomics/Project_1/dataset2_python/scripts
